In [1]:
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F

创建一个经典的LeNet网络

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
device

device(type='cpu')

In [4]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        # 1: 图像的输入通道(1是黑白图像), 6: 输出通道, 3x3: 卷积核的尺寸
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5x5 是经历卷积操作后的图片尺寸
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, int(x.nelement() / x.shape[0]))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = LeNet().to(device=device)

In [5]:
module = model.conv1
print(list(module.named_parameters()))

[('weight', Parameter containing:
tensor([[[[-0.3249,  0.0024,  0.1380],
          [ 0.3285,  0.0024,  0.2320],
          [ 0.3222,  0.0152, -0.0489]]],


        [[[-0.0894,  0.1978, -0.1947],
          [-0.0299, -0.0440,  0.2470],
          [-0.0769, -0.1188, -0.2219]]],


        [[[ 0.2724, -0.1438, -0.2627],
          [ 0.2301, -0.0314, -0.0930],
          [-0.0545,  0.1504, -0.0504]]],


        [[[ 0.0624, -0.0672, -0.1851],
          [ 0.2783,  0.2560,  0.2220],
          [-0.2856, -0.0847, -0.0672]]],


        [[[ 0.1904, -0.2472,  0.1982],
          [-0.2487, -0.0719,  0.0186],
          [ 0.1364,  0.0674,  0.2810]]],


        [[[ 0.0624,  0.1727,  0.3053],
          [-0.1114,  0.0935, -0.3089],
          [-0.3035, -0.1825,  0.1328]]]], requires_grad=True)), ('bias', Parameter containing:
tensor([-0.0134,  0.1742,  0.0337,  0.2687,  0.2862,  0.1170],
       requires_grad=True))]


In [6]:
print(list(module.named_buffers()))

[]


In [7]:
# 第一个参数: module, 代表要进行剪枝的特定模块, 之前我们已经制定了module=model.conv1,
#             说明这里要对第一个卷积层执行剪枝.
# 第二个参数: name, 指定要对选中的模块中的哪些参数执行剪枝.
#             这里设定为name="weight", 意味着对连接网络中的weight剪枝, 而不对bias剪枝.
# 第三个参数: amount, 指定要对模型中多大比例的参数执行剪枝.
#             amount是一个介于0.0-1.0的float数值, 或者一个正整数指定剪裁掉多少条连接边.

prune.random_unstructured(module, name="weight", amount=0.3)

Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))

In [9]:
print(list(module.named_parameters()))

[('bias', Parameter containing:
tensor([-0.0134,  0.1742,  0.0337,  0.2687,  0.2862,  0.1170],
       requires_grad=True)), ('weight_orig', Parameter containing:
tensor([[[[-0.3249,  0.0024,  0.1380],
          [ 0.3285,  0.0024,  0.2320],
          [ 0.3222,  0.0152, -0.0489]]],


        [[[-0.0894,  0.1978, -0.1947],
          [-0.0299, -0.0440,  0.2470],
          [-0.0769, -0.1188, -0.2219]]],


        [[[ 0.2724, -0.1438, -0.2627],
          [ 0.2301, -0.0314, -0.0930],
          [-0.0545,  0.1504, -0.0504]]],


        [[[ 0.0624, -0.0672, -0.1851],
          [ 0.2783,  0.2560,  0.2220],
          [-0.2856, -0.0847, -0.0672]]],


        [[[ 0.1904, -0.2472,  0.1982],
          [-0.2487, -0.0719,  0.0186],
          [ 0.1364,  0.0674,  0.2810]]],


        [[[ 0.0624,  0.1727,  0.3053],
          [-0.1114,  0.0935, -0.3089],
          [-0.3035, -0.1825,  0.1328]]]], requires_grad=True))]


In [10]:
print(list(module.named_buffers()))

[('weight_mask', tensor([[[[0., 1., 1.],
          [1., 1., 0.],
          [0., 1., 1.]]],


        [[[1., 1., 1.],
          [1., 0., 1.],
          [0., 1., 0.]]],


        [[[1., 0., 0.],
          [1., 0., 0.],
          [1., 1., 1.]]],


        [[[1., 1., 0.],
          [0., 1., 0.],
          [1., 1., 1.]]],


        [[[1., 0., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]],


        [[[1., 0., 1.],
          [1., 0., 1.],
          [1., 1., 1.]]]]))]


结论: 模型经历剪枝操作后, 原始的权重矩阵weight参数不见了, 变成了weight_orig. 并且刚刚打印为空列表的module.named_buffers(), 此时拥有了一个weight_mask参数.

这时打印module.weight属性值, 看看有什么启发?

In [11]:
print(module.weight)

tensor([[[[-0.0000,  0.0024,  0.1380],
          [ 0.3285,  0.0024,  0.0000],
          [ 0.0000,  0.0152, -0.0489]]],


        [[[-0.0894,  0.1978, -0.1947],
          [-0.0299, -0.0000,  0.2470],
          [-0.0000, -0.1188, -0.0000]]],


        [[[ 0.2724, -0.0000, -0.0000],
          [ 0.2301, -0.0000, -0.0000],
          [-0.0545,  0.1504, -0.0504]]],


        [[[ 0.0624, -0.0672, -0.0000],
          [ 0.0000,  0.2560,  0.0000],
          [-0.2856, -0.0847, -0.0672]]],


        [[[ 0.1904, -0.0000,  0.1982],
          [-0.2487, -0.0719,  0.0186],
          [ 0.1364,  0.0674,  0.2810]]],


        [[[ 0.0624,  0.0000,  0.3053],
          [-0.1114,  0.0000, -0.3089],
          [-0.3035, -0.1825,  0.1328]]]], grad_fn=<MulBackward0>)


结论: 经过剪枝操作后的模型, 原始的参数存放在了weight_orig中, 对应的剪枝矩阵存放在weight_mask中, 而将weight_mask视作掩码张量, 再和weight_orig相乘的结果就存放在了weight中.

注意: 剪枝操作后的weight已经不再是module的参数(parameter), 而只是module的一个属性(attribute).

对于每一次剪枝操作, 模型都会对应一个具体的_forward_pre_hooks函数用于剪枝.

In [12]:
print(module._forward_pre_hooks)

OrderedDict([(0, <torch.nn.utils.prune.RandomUnstructured object at 0x000001A1BFDC8FD0>)])


In [13]:
# 第一个参数: module, 代表剪枝的对象, 此处代表LeNet中的conv1
# 第二个参数: name, 代表剪枝对象中的具体参数, 此处代表偏置量
# 第三个参数: amount, 代表剪枝的数量, 可以设置为0.0-1.0之间表示比例, 也可以用正整数表示剪枝的参数绝对数量
prune.l1_unstructured(module, name="bias", amount=3)

# 再次打印模型参数
print(list(module.named_parameters()))
print('*'*50)
print(list(module.named_buffers()))
print('*'*50)
print(module.bias)
print('*'*50)
print(module._forward_pre_hooks)

[('weight_orig', Parameter containing:
tensor([[[[-0.3249,  0.0024,  0.1380],
          [ 0.3285,  0.0024,  0.2320],
          [ 0.3222,  0.0152, -0.0489]]],


        [[[-0.0894,  0.1978, -0.1947],
          [-0.0299, -0.0440,  0.2470],
          [-0.0769, -0.1188, -0.2219]]],


        [[[ 0.2724, -0.1438, -0.2627],
          [ 0.2301, -0.0314, -0.0930],
          [-0.0545,  0.1504, -0.0504]]],


        [[[ 0.0624, -0.0672, -0.1851],
          [ 0.2783,  0.2560,  0.2220],
          [-0.2856, -0.0847, -0.0672]]],


        [[[ 0.1904, -0.2472,  0.1982],
          [-0.2487, -0.0719,  0.0186],
          [ 0.1364,  0.0674,  0.2810]]],


        [[[ 0.0624,  0.1727,  0.3053],
          [-0.1114,  0.0935, -0.3089],
          [-0.3035, -0.1825,  0.1328]]]], requires_grad=True)), ('bias_orig', Parameter containing:
tensor([-0.0134,  0.1742,  0.0337,  0.2687,  0.2862,  0.1170],
       requires_grad=True))]
**************************************************
[('weight_mask', tensor([[[[0., 1.,

结论: 在module的不同参数集合上应用不同的剪枝策略, 我们发现模型参数中不仅仅有了weight_orig, 也有了bias_orig. 在起到掩码张量作用的named_buffers中, 也同时出现了weight_mask和bias_mask. 最后, 因为我们在两类参数上应用了两种不同的剪枝函数, 因此_forward_pre_hooks中也打印出了2个不同的函数结果.

### 序列化一个剪枝模型